# Part 1 : Checking the data and defining Extract, Transform, Load steps

In [1]:
import numpy as np
import pandas as pd
import logging # Logging and the connected configuration will be used to pop up some messages to see if our extraction works well

logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)

In [3]:
def extract(file_path):
    return pd.read_json(file_path) #Files are loaded in my personal computer, so I'll be using their path

In [52]:
#Let's check the data before we start transforming it
raw=pd.read_json("C:/Users/New PC 3/Desktop/Practice PR/Python 1.1 - Spotify ETL ELT test/stream_history/2023-2024.json")
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16172 entries, 0 to 16171
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ts                                 16172 non-null  object 
 1   platform                           16172 non-null  object 
 2   ms_played                          16172 non-null  int64  
 3   conn_country                       16172 non-null  object 
 4   ip_addr                            16172 non-null  object 
 5   master_metadata_track_name         16170 non-null  object 
 6   master_metadata_album_artist_name  16170 non-null  object 
 7   master_metadata_album_album_name   16170 non-null  object 
 8   spotify_track_uri                  16170 non-null  object 
 9   episode_name                       2 non-null      object 
 10  episode_show_name                  2 non-null      object 
 11  spotify_episode_uri                2 non-null      obj

In [ ]:
#We have some confidential information about country(conn_country) and ip address(ip_addr), both are also irrelevant to our analysis.

In [5]:
def transform(raw_data):
    #We start by handling time and date issues. Our current date is in object format. We have to transform it.
    #Then we extract date and time information from it. 

        #Turning ts columns' data type from object to datetime
    raw_data["ts"]=pd.to_datetime(raw_data["ts"])
        #Extracting date and time information and creating new columns
    raw_data["date"]=raw_data["ts"].dt.date
    raw_data["time"]=raw_data["ts"].dt.time
        #When we're done with ts column, we can drop it.
    raw_data=raw_data.drop("ts", axis=1)
    
    #Dropping some of the columns that include confidential information.
    raw_data=raw_data.drop(['conn_country','ip_addr', 'spotify_track_uri'], axis=1)
    
    #Some of our columns have unpractically long names to use. We can alter them before we use them.
        #Renaming columns
    raw_data=raw_data.rename(columns={"master_metadata_track_name":"track_name", "master_metadata_album_artist_name":"artist_name", 
                                      "master_metadata_album_album_name":"album_name"})

    #Our dataset holds both music and podcast information. We have to filter the relevant data, the music data first.
        #Seperating podcast and song data
    raw_data=raw_data[raw_data["episode_name"].isnull()] # With this code, you can filter them by episode name being null.
    # If you change isnull() part to notnull(), you can get podcast data as well. In this example, we'll be focusing on music data.

    #Dropping the remaining uneccessary columns
    raw_data=raw_data.drop(['episode_name', 'episode_show_name','spotify_episode_uri','audiobook_title', 'audiobook_uri','audiobook_chapter_uri', 
                            'audiobook_chapter_title','offline_timestamp', 'incognito_mode'], axis=1)
    return raw_data

In [7]:
# Defining a function to load our data into a csv file. 
def load(cleaned_music_data, file_path):
    cleaned_music_data.to_csv(file_path, index=False)
    print(f"Successfully loaded data to {file_path}")

In [58]:
# Now, we have all of the functions we need to run our ETL process
raw_music_data=extract("C:/Users/New PC 3/Desktop/Practice PR/Python 1.1 - Spotify ETL ELT test/stream_history/2023-2024.json")
cleaned_music_data=transform(raw_music_data)
load(cleaned_music_data, "ETLtest.csv")

Successfully loaded data to ETLtest.csv


In [60]:
# Let's check the dataset we got after the process
data_check=pd.read_csv("ETLtest.csv")
data_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16170 entries, 0 to 16169
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   platform      16170 non-null  object
 1   ms_played     16170 non-null  int64 
 2   track_name    16170 non-null  object
 3   artist_name   16170 non-null  object
 4   album_name    16170 non-null  object
 5   reason_start  16170 non-null  object
 6   reason_end    16170 non-null  object
 7   shuffle       16170 non-null  bool  
 8   skipped       16170 non-null  bool  
 9   offline       16170 non-null  bool  
 10  date          16170 non-null  object
 11  time          16170 non-null  object
dtypes: bool(3), int64(1), object(8)
memory usage: 1.2+ MB


# Test 2 : ETL functioning

In [ ]:
# We have 5 different files holding music data.
# Logical way is to write another function that will take all the files, transform and get them together.

In [16]:
import glob #glob will help us to use the path names by catching patterns

In [18]:
def ETL(files, load_file):
    all_data = []

    for file in files:
        raw_data=extract(file)
        clean_data=transform(raw_data)
        all_data.append(clean_data) #We'll clean all the data and concat them into one single dataframe

    combined_data=pd.concat(all_data, ignore_index=True)

    load(combined_data, load_file)

In [20]:
#As all of our data is in a single file called stream_history, we can pass *.json to get all json files in that folder.
json_files=glob.glob("C:/Users/New PC 3/Desktop/Practice PR/Python 1.1 - Spotify ETL ELT test/stream_history/*.json")
ETL(json_files, "ETLplease.csv")

Successfully loaded data to ETLplease.csv


In [22]:
full=pd.read_csv("ETLplease.csv")
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65063 entries, 0 to 65062
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   platform      65063 non-null  object
 1   ms_played     65063 non-null  int64 
 2   track_name    65063 non-null  object
 3   artist_name   65063 non-null  object
 4   album_name    65063 non-null  object
 5   reason_start  65063 non-null  object
 6   reason_end    65063 non-null  object
 7   shuffle       65063 non-null  bool  
 8   skipped       65063 non-null  bool  
 9   offline       65063 non-null  bool  
 10  date          65063 non-null  object
 11  time          65063 non-null  object
dtypes: bool(3), int64(1), object(8)
memory usage: 4.7+ MB
